In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import requests

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { max-width:100% !important; }</style>"))

In [3]:
base_url = 'https://api.binance.com' 
def get_base_currencies():
    endpoint = '/api/v3/exchangeInfo'
    url = base_url + endpoint
    response = requests.get(url)

    if response.status_code == 200:
        exchange_info = response.json()
        symbols = exchange_info['symbols']
        tradeable = list()
        base_currencies = set()
        for symbol in symbols:
            if symbol["status"]=='TRADING' and symbol["isSpotTradingAllowed"]:
                base_currencies.add(symbol['baseAsset'])
                tradeable.append(symbol)

        return sorted(list(base_currencies)),symbols,tradeable
    else:
        print(f'Error: {response.status_code}')
        return None,symbols,tradeable

base_currencies,symbols,tradeable = get_base_currencies()
#print(base_currencies)
tradeable_tickers = [tt["symbol"] for tt in tradeable]
len(tradeable_tickers)

1403

In [4]:
print(len(tradeable_tickers),tradeable_tickers[:10])

1403 ['ETHBTC', 'LTCBTC', 'BNBBTC', 'NEOBTC', 'QTUMETH', 'EOSETH', 'SNTETH', 'BNTETH', 'GASBTC', 'BNBETH']


In [5]:

basetickers = ["USDT","ETH","BTC","BNB","LTC"]
def get_high_volume_trading_pairs():
    endpoint = '/api/v3/ticker/24hr'
    url = base_url + endpoint
    response = requests.get(url)

    if response.status_code == 200:
        trading_pairs_data = response.json()
        trading_pairs = sorted(trading_pairs_data, key=lambda x: float(x['quoteVolume']), reverse=True)
        print("all pairs",len(trading_pairs))
        trading_pairs = [sp for sp in trading_pairs if sp["symbol"] in tradeable_tickers]
        print("spot tradeable pairs",len(trading_pairs))
        trading_pairs = [sp for sp in trading_pairs if any(sp["symbol"].endswith(bt) for bt in basetickers)]
        print("with base pairs",len(trading_pairs))
        high_volume_pairs = []
        for pair in trading_pairs[:10]:
            high_volume_pairs.append({
                'symbol': pair['symbol'],
                'quote_volume': float(pair['quoteVolume'])
            })

        return high_volume_pairs,trading_pairs
    else:
        print(f'Error: {response.status_code}')
        return None,trading_pairs

high_volume_trading_pairs,trading_pairs = get_high_volume_trading_pairs() 

all pairs 2203
spot tradeable pairs 1403
with base pairs 819
